In [2]:
"""
@author: OMIS6000 (Winter2024) Group 12
"""

from gurobipy import GRB
import gurobipy as gb
import pandas as pd

# Create the optimization model
model = gb.Model("FIBA Selection")

ability = pd.read_csv('https://drive.google.com/uc?id=1_vOXTRV68lrCs_Q1c0zcQ21Q_q8AT6FK&export=download?usp=sharing', index_col='Number')

# Create the one class of eight decision variables 
x = model.addVars(150, vtype=GRB.BINARY, name="Player")

# The objective function
model.setObjective(gb.quicksum(((ability.iloc[i,1:8]).sum())*x[i] for i in range(150)), GRB.MAXIMIZE)

# Add the constraints

# Capacity constraints
model.addConstr(gb.quicksum(x[i] for i in range(150)) == 21)

# Position constraints
index_guard = ability[(ability['Position'] == 'G') | (ability['Position'] == 'G/F')].index.tolist()
model.addConstr(gb.quicksum(x[i-1] for i in index_guard) >= (21 * 0.3), "Guard position")

index_fc = ability[(ability['Position'] == 'F') | (ability['Position'] == 'C') | (ability['Position'] == 'F/C')].index.tolist()
model.addConstr(gb.quicksum(x[i-1] for i in index_fc) >= (21 * 0.4), "Guard position")

# Average score constraints
for j in range(7):
    model.addConstr(((gb.quicksum(x[i]*ability.iloc[i,j+1] for i in range(150)) / 21) >= 2.05), "average score")

# Constraint: if any 20-24, not all 72-78
for i in range(19,24):
    for j in range(71,78):
        model.addConstr(x[i] <= 1 - x[j], "If any 20-24, not all 72-78")

# Constraint: if any 105-114, at least one 45-49 and at least one 60-69
for i in range(104,114):
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44,49)), "If any 105-114, at least 45-49")
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64,69)), "If any 105-114, at least 65-69")

# at least one player from every 10 players
for i in range(14):
    model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, "at least one player")
    
# Optimally solve the problem
model.optimize()

# Print the objective and decision variables
model.printAttr('X')

# The status of the model
print("Model Status: ", model.status)

selected = [i for i, var in enumerate(model.getVars()) if var.x !=0]
count = ability.iloc[selected]['Position'].value_counts()
guards = count.loc['G'] + count.loc['G/F']

print(f'Number of G and G/F: {guards}')

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 79 rows, 150 columns and 1680 nonzeros
Model fingerprint: 0x766b47ce
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 79 rows, 150 columns, 1341 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.610000e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     361.0000000  361.00000  0.00%     -    0s

Explored 1 nodes (21 simplex iterations) 

In [3]:

def smallest_invitation(number):
    # Create the optimization model
    model = gb.Model("FIBA Selection")

    # Create the one class of eight decision variables 
    x = model.addVars(150, vtype=GRB.BINARY, name="Player")

    # The objective function
    model.setObjective(gb.quicksum(((ability.iloc[i,1:8]).sum())*x[i] for i in range(150)), GRB.MAXIMIZE)

    # Add the constraints

    # Capacity constraints
    model.addConstr(gb.quicksum(x[i] for i in range(150)) == number, "capacity constraint")

    # Position constraints
    index_guard = ability[(ability['Position'] == 'G') | (ability['Position'] == 'G/F')].index.tolist()
    model.addConstr(gb.quicksum(x[i-1] for i in index_guard) >= (number * 0.3), "Guard position")

    index_fc = ability[(ability['Position'] == 'F') | (ability['Position'] == 'C') | (ability['Position'] == 'F/C')].index.tolist()
    model.addConstr(gb.quicksum(x[i-1] for i in index_fc) >= (number * 0.4), "Guard position")

    # Average score constraints
    for j in range(7):
        model.addConstr(((gb.quicksum(x[i]*ability.iloc[i,j+1] for i in range(150)) / number) >= 2.05), "average score")

    # Constraint: if any 20-24, not all 72-78
    for i in range(19,24):
        for j in range(71,78):
            model.addConstr(x[i] <= 1 - x[j], "If any 20-24, not all 72-78")

    # Constraint: if any 105-114, at least one 45-49 and at least one 60-69
    for i in range(104,114):
        model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44,49)), "If any 105-114, at least 45-49")
        model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64,69)), "If any 105-114, at least 65-69")

    # at least one player from every 10 players
    for i in range(14):
        model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, "at least one player")
    
    return model

# Initialize the smallest number of invitations
number_of_invitations = 21

# Iterate to find the smallest number of invitations before infeasibility
while True:
    # Set the number of invitations to the current value
    model = smallest_invitation(number_of_invitations)
    
    # Solve the model
    model.optimize()
    
    # Check feasibility status
    if model.status != GRB.Status.OPTIMAL:
        # Infeasible solution found, break the loop
        break
    
    # Increment the number of invitations for the next iteration
    number_of_invitations -= 1

# Print the result
print("Smallest number of invitations before infeasibility:", number_of_invitations+1)

model.computeIIS()
infeasible_constraints = [constr.ConstrName for constr in model.getConstrs() if constr.IISConstr]
print("Constraints leading to infeasibility:", infeasible_constraints)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 79 rows, 150 columns and 1680 nonzeros
Model fingerprint: 0x766b47ce
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 79 rows, 150 columns, 1341 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.610000e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     361.0000000  361.00000  0.00%     -    0s

Explored 1 nodes (21 simplex iterations) 

In [4]:

# Print the result
print("Smallest number of invitations before infeasibility:", number_of_invitations+1)

model.computeIIS()
infeasible_constraints = [constr.ConstrName for constr in model.getConstrs() if constr.IISConstr]
print("Constraints leading to infeasibility:", infeasible_constraints)

Smallest number of invitations before infeasibility: 14
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads


IIS computed: 15 constraints, 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
Constraints leading to infeasibility: ['capacity constraint', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player', 'at least one player']
